## Prediction de salaire

### Bibliothèques utiles

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

pd.set_option("display.float_format", lambda x: f"{x:,.3f}")
sns.set_context("talk")
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)


### I-  Description de la base de donnée

#### Salaires dans le secteur privé selon le sexe et la catégorie socioprofessionnelle (base communale)

* Le champ correspond aux salariés du privé, y compris bénéficiaires de contrats aidés et de contrats de professionnalisation ; hors apprentis, stagiaires, salariés agricoles et salariés des particuliers employeurs.

* Les données sur les salaires au lieu de travail sont ventilées selon le sexe et la catégorie socioprofessionnelle (hors agriculture), et détaillées par territoire : commune, arrondissement municipal, arrondissement, aire d'attraction des villes 2020, bassin de vie 2022, établissement public de coopération intercommunal, unité urbaine 2020, zone d'emploi 2020, département, région, France hors Mayotte.

Variables explicatives retenues :

SEX : Sexe (Homme/Femme) - impact attendu sur l'écart salarial

PCS_ESE : Profession et Catégorie Socioprofessionnelle - déterminant principal

TIME_PERIOD : Année (2022-2023) - évolution temporelle

GEO : Code géographique - variations territoriales

In [48]:
## lien vers le dataset
data_path = r"dataset\DS_BTS_SAL_EQTP_SEX_PCS_2023_data.csv"
metadata_path = r"dataset\DS_BTS_SAL_EQTP_SEX_PCS_2023_metadata.csv"


In [ ]:
data = pd.read_csv(data_path, sep=';')

In [50]:
# Affichage des informations de base
print(f" Dimensions du dataset: {data.shape}")
print(f" Colonnes disponibles: {list(data.columns)}")

 Dimensions du dataset: (370710, 9)
 Colonnes disponibles: ['GEO', 'GEO_OBJECT', 'FREQ', 'SEX', 'PCS_ESE', 'DERA_MEASURE', 'CONF_STATUS', 'TIME_PERIOD', 'OBS_VALUE']


In [51]:
print("\n Aperçu des premières lignes:")
print(data.head())


 Aperçu des premières lignes:
     GEO GEO_OBJECT FREQ SEX PCS_ESE                      DERA_MEASURE CONF_STATUS  TIME_PERIOD  OBS_VALUE
0  26362     BV2022    A   F      _T  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2022  2,157.285
1  26324     BV2022    A  _T       4  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2022  3,112.938
2  26307     BV2022    A  _T       6  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2023  2,013.097
3  26362     BV2022    A  _T      _T  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2023  2,483.037
4  27170     BV2022    A   F       4  SALAIRE_NET_EQTP_MENSUEL_MOYENNE           F         2022  2,107.221


In [52]:
print(" Types de données:")
print(data.dtypes)

 Types de données:
GEO              object
GEO_OBJECT       object
FREQ             object
SEX              object
PCS_ESE          object
DERA_MEASURE     object
CONF_STATUS      object
TIME_PERIOD       int64
OBS_VALUE       float64
dtype: object


In [53]:

print("\n📋 Informations détaillées:")
print(data.info())


📋 Informations détaillées:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370710 entries, 0 to 370709
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   GEO           370710 non-null  object 
 1   GEO_OBJECT    370710 non-null  object 
 2   FREQ          370710 non-null  object 
 3   SEX           370710 non-null  object 
 4   PCS_ESE       370710 non-null  object 
 5   DERA_MEASURE  370710 non-null  object 
 6   CONF_STATUS   370710 non-null  object 
 7   TIME_PERIOD   370710 non-null  int64  
 8   OBS_VALUE     350250 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 25.5+ MB
None


In [54]:
# Analyse des variables uniques
print("📊 Analyse des variables catégorielles:")
categorical_vars = ['GEO_OBJECT', 'FREQ', 'SEX', 'PCS_ESE', 'DERA_MEASURE', 'CONF_STATUS']

for var in categorical_vars:
    unique_vals = data[var].unique()
    print(f"\n{var} ({len(unique_vals)} valeurs uniques):")
    print(f"  Valeurs: {unique_vals[:10]}...")  # Afficher les 10 premières valeurs

print(f"\n📈 Variable cible (OBS_VALUE) - Statistiques descriptives:")
print(data['OBS_VALUE'].describe())

print(f"\n📅 Période temporelle:")
print(f"  Années disponibles: {sorted(data['TIME_PERIOD'].unique())}")

print(f"\n❌ Valeurs manquantes:")
missing_data = data.isnull().sum()
print(missing_data[missing_data > 0])


📊 Analyse des variables catégorielles:

GEO_OBJECT (11 valeurs uniques):
  Valeurs: ['BV2022' 'ARR' 'AAV2020' 'COM' 'UU2020' 'ARM' 'EPCI' 'ZE2020' 'DEP'
 'FRANCE']...

FREQ (1 valeurs uniques):
  Valeurs: ['A']...

SEX (3 valeurs uniques):
  Valeurs: ['F' '_T' 'M']...

PCS_ESE (5 valeurs uniques):
  Valeurs: ['_T' '4' '6' '5' '1T3']...

DERA_MEASURE (1 valeurs uniques):
  Valeurs: ['SALAIRE_NET_EQTP_MENSUEL_MOYENNE']...

CONF_STATUS (2 valeurs uniques):
  Valeurs: ['F' 'C']...

📈 Variable cible (OBS_VALUE) - Statistiques descriptives:
count   350,250.000
mean      2,445.223
std         758.391
min         793.399
25%       1,908.283
50%       2,191.887
75%       2,671.732
max      14,047.315
Name: OBS_VALUE, dtype: float64

📅 Période temporelle:
  Années disponibles: [np.int64(2022), np.int64(2023)]

❌ Valeurs manquantes:
OBS_VALUE    20460
dtype: int64


In [57]:
data['GEO_OBJECT'].unique()

array(['BV2022', 'ARR', 'AAV2020', 'COM', 'UU2020', 'ARM', 'EPCI',
       'ZE2020', 'DEP', 'FRANCE', 'REG'], dtype=object)

SEX
F     123570
_T    123570
M     123570
Name: count, dtype: int64

### II-  Analyse exploratoire de donnée

#### A analyse univarié

#### B Analyse bivariée

### III- Modèle de prédiction 

#### A nettoyage de donnée

#### Modélisation de donnée

In [68]:
df = pd.read_csv(data_path, sep=';')

In [69]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor


In [70]:
# Séparer X et y
X = df.drop(columns=["OBS_VALUE", "FREQ", "DERA_MEASURE"])
y = df["OBS_VALUE"]

In [71]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370710 entries, 0 to 370709
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   GEO          370710 non-null  object
 1   GEO_OBJECT   370710 non-null  object
 2   SEX          370710 non-null  object
 3   PCS_ESE      370710 non-null  object
 4   CONF_STATUS  370710 non-null  object
 5   TIME_PERIOD  370710 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 17.0+ MB


In [75]:
cat_features = ["GEO_OBJECT", "SEX", "PCS_ESE", "CONF_STATUS","TIME_PERIOD"]
num_features = ["GEO"]

In [76]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Supposons que df soit ton DataFrame
X = df.drop(columns=["OBS_VALUE", "FREQ", "DERA_MEASURE"])
y = df["OBS_VALUE"]

# Colonnes par type
onehot_cols = ["GEO_OBJECT", "SEX"]
target_cols = ["PCS_ESE"]
binary_cols = ["CONF_STATUS"]
num_cols = ["Year"]  # suppose que tu as une colonne Year

# Encoders
onehot_enc = OneHotEncoder(drop="first", sparse_output=False)
target_enc = TargetEncoder()
scaler = StandardScaler()

# Préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ("onehot", onehot_enc, onehot_cols),
        ("target", target_enc, target_cols),
        ("binary", "passthrough", binary_cols),  # ou OrdinalEncoder
        ("num", scaler, num_cols),
    ]
)

# Pipeline finale
pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    # Ici tu peux ajouter un modèle : ("model", Regression/ML)
])

# Fit-transform
X_processed = pipeline.fit_transform(X, y)


ValueError: A given column is not a column of the dataframe